In [1]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import scale

In [2]:
features = pd.read_csv('features/features.csv', index_col='match_id')
features.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

features_test = pd.read_csv('features_test/features_test.csv', index_col='match_id')

f_win = pd.read_csv('features/features.csv', index_col='match_id')['radiant_win'].fillna(0)

In [3]:
def XY_tt(features, features_test):
    X_train , Y_train = features.fillna(0).as_matrix().astype(float), f_win.as_matrix().astype(float)
    X_test = features_test.fillna(0).as_matrix().astype(float)
    return scale(X_train), Y_train, scale(X_test)
X_train , Y_train, X_test = XY_tt(features, features_test)

C:\Users\Denis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [4]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

1.Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [5]:
def Testing_grid(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):
    X_train = scale(X_train)
    
    grid = {'C': c}
    clf = LogisticRegression(penalty='l2')
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='roc_auc', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return c[np.argmax(means)], max(means)
    
def Testing_cross(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):     
    start_time = datetime.datetime.now()
    scores=[]
    X_train = scale(X_train)
    for x in c:
        #print (x)
        scores.append(np.array(cross_val_score(LogisticRegression(C=x), X_train, Y_train, scoring='roc_auc', cv=cv)))
    scores=np.array(scores)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    print (c[np.argmax([np.mean(sc) for sc in scores])])
    return c[np.argmax([np.mean(sc) for sc in scores])], max([np.mean(sc) for sc in scores])

In [6]:
Testing_grid(X_train, Y_train)

Time elapsed: 0:02:52.302299
0.687 (+/-0.003) for {'C': 1e-09}
0.687 (+/-0.003) for {'C': 1e-08}
0.687 (+/-0.003) for {'C': 1e-07}
0.688 (+/-0.003) for {'C': 1e-06}
0.695 (+/-0.003) for {'C': 1e-05}
0.711 (+/-0.003) for {'C': 0.0001}
0.716 (+/-0.003) for {'C': 0.001}
0.717 (+/-0.002) for {'C': 0.01}
0.717 (+/-0.002) for {'C': 0.1}
0.717 (+/-0.002) for {'C': 1.0}
0.717 (+/-0.002) for {'C': 10.0}
0.717 (+/-0.002) for {'C': 100.0}
0.717 (+/-0.002) for {'C': 1000.0}
0.717 (+/-0.002) for {'C': 10000.0}
0.717 (+/-0.002) for {'C': 100000.0}
0.717 (+/-0.002) for {'C': 1000000.0}
0.717 (+/-0.002) for {'C': 10000000.0}


(0.01, 0.71655026972591407)

Наилучший вариант параметра C = 0.01, при максимальном качестве в 0.71655026972591407. Работает он сильно быстрее, 2 минуты против 4 часов на бустинге, при чуть большем качестве 0.71743541511155817. Очевидно, что бустинг на композиции алгоритмов работает дольше, но зависит от глубины и кол-ва деревьев.

2.Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [28]:
def Dell_cat(data_r):
    data = data_r.copy()
    
    todell=[]
    
    for ind, name in enumerate(data.columns):
        if len(data[name].unique())<150 and name.split('_')[1] in ['hero', 'type', 'xp', 'lh']:
            #print (name, ind, len(data[name].unique()))
            todell.append(name)
    return data.drop(todell, axis=1)

In [29]:
X_train , Y_train, X_test = XY_tt(Dell_cat(features), Dell_cat(features_test))

C:\Users\Denis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [30]:
Testing_grid(X_train, Y_train)

Time elapsed: 0:02:14.948020
0.679 (+/-0.003) for {'C': 1e-09}
0.679 (+/-0.003) for {'C': 1e-08}
0.679 (+/-0.003) for {'C': 1e-07}
0.680 (+/-0.003) for {'C': 1e-06}
0.686 (+/-0.003) for {'C': 1e-05}
0.704 (+/-0.003) for {'C': 0.0001}
0.714 (+/-0.003) for {'C': 0.001}
0.715 (+/-0.003) for {'C': 0.01}
0.715 (+/-0.003) for {'C': 0.1}
0.715 (+/-0.003) for {'C': 1.0}
0.715 (+/-0.003) for {'C': 10.0}
0.715 (+/-0.003) for {'C': 100.0}
0.715 (+/-0.003) for {'C': 1000.0}
0.715 (+/-0.003) for {'C': 10000.0}
0.715 (+/-0.003) for {'C': 100000.0}
0.715 (+/-0.003) for {'C': 1000000.0}
0.715 (+/-0.003) for {'C': 10000000.0}


(0.1, 0.71513701451769263)

Провел быстрые испытания по признакам содежащим 'level', 'item','lh' это ухудшило предсказания. Но если убрать только - 'hero' и 'type', то очевидно, что это практически не повлияет на качество алгоритма

3.На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, 

In [10]:
def Compare(features, f_win):
    data = features.copy()
    data['radiant_win'] = f_win
    h_col = list(filter(lambda name: name.split('_')[1]=='hero' ,data.columns))
    results=[]
    for player in h_col:
        #for match in data.index:
        for hero in data[player].unique():
            if player.split('_')[0] in ['r1', 'r2', 'r3', 'r4', 'r5']:
                results.append(np.array([hero, player.split('_')[0] ,int(data[(data[player] == hero) & (data['radiant_win']==1)]['radiant_win'].count())]))
            else:
                results.append(np.array([hero, player.split('_')[0] ,int(data[(data[player] == hero) & (data['radiant_win']==0)]['radiant_win'].count())]))
    results = pd.DataFrame(results, columns=['Hero', 'Player', 'Win_count'])
    results['Win_count']= results['Win_count'].astype(int)
    return results.sort_values(by=['Win_count'], ascending = False)

In [11]:
tmp = Compare(features, f_win)
tmp.head(15)

,Hero,Player,Win_count
0,11,r1,2795
479,7,r5,2395
135,11,r2,2367
335,7,r4,2366
222,7,r3,2233
136,7,r2,2172
980,7,d5,2130
865,7,d4,2105
783,7,d3,2076
233,11,r3,2073


Если выбирать из top15 по количеству побед, можно заметить, что чаще всего выигрывают за 11 и 7 героя. Хотя, если убрать привзяку к игроку, то можно получить следующий результат.

In [12]:
tmp[['Hero', 'Win_count']].groupby(['Hero']).sum().sort_values('Win_count', ascending = False).head()

,Win_count
Hero,
7,21306
11,19766
112,17912
50,17662
72,17617


Наши ожидания подтвердились, также можно заметить и других героев, игра с которомы ведет к победе.

4.Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [15]:
def Bag_words(features):
    data = features.copy()
    
    N = data['r1_hero'].max()
    
    # N — количество различных героев в выборке
    X_pick = np.zeros((data.shape[0], N))
    
    for i, match_id in enumerate(data.index):
        for p in range(1, 6):
            X_pick[i, data.loc[match_id, 'r%d_hero' % p]-1] = 1
            X_pick[i, data.loc[match_id, 'd%d_hero' % p]-1] = -1
    results = pd.DataFrame(X_pick, columns=['Bhero_'+str(i) for i in range(1, N+1)], index = data.index)
    return pd.concat([data, results], axis = 1)

In [16]:
X_train , Y_train, X_test = XY_tt(Dell_cat(Bag_words(features)), Dell_cat(Bag_words(features_test)))

C:\Users\Denis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


5.Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [17]:
Testing_grid(X_train, Y_train)

Time elapsed: 0:04:44.954659
0.700 (+/-0.003) for {'C': 1e-09}
0.700 (+/-0.003) for {'C': 1e-08}
0.701 (+/-0.003) for {'C': 1e-07}
0.702 (+/-0.003) for {'C': 1e-06}
0.715 (+/-0.003) for {'C': 1e-05}
0.743 (+/-0.004) for {'C': 0.0001}
0.752 (+/-0.005) for {'C': 0.001}
0.752 (+/-0.005) for {'C': 0.01}
0.752 (+/-0.005) for {'C': 0.1}
0.752 (+/-0.005) for {'C': 1.0}
0.752 (+/-0.005) for {'C': 10.0}
0.752 (+/-0.005) for {'C': 100.0}
0.752 (+/-0.005) for {'C': 1000.0}
0.752 (+/-0.005) for {'C': 10000.0}
0.752 (+/-0.005) for {'C': 100000.0}
0.752 (+/-0.005) for {'C': 1000000.0}
0.752 (+/-0.005) for {'C': 10000000.0}


(0.01, 0.75196404316175602)

Можно заметить заметное улучшение качества. Раньше информация о героях не несла никакого смысла, теперь каждый герой осмысленее.

6.Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [31]:
clf = LogisticRegression(penalty='l2', C=0.01)
clf.fit(X_train, Y_train)
win = clf.predict_proba(X_test)

In [32]:
pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win']).to_csv('result.scv', index=False)

Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [37]:
print('Max', pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win'])['radiant_win'].max())
print('Min', pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win'])['radiant_win'].min())

Max 0.98260417143
Min 0.00336830843809


Score 0.75522

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
clf = GradientBoostingClassifier(n_estimators=2, max_depth=4)
clf.fit(X_train, Y_train)
win = clf.predict_proba(X_test)

In [24]:
pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win']).to_csv('result.scv', index=False)

Был интерес, но Score 0.62232.